In [1]:
# 필요 라이브러리 import 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import warnings
warnings.filterwarnings(action='ignore') 

In [4]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_classification
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

In [5]:
train = pd.read_csv("./data/8-1 train(std).csv",index_col=0)
test1 = pd.read_csv("./data/8-1 test1(std).csv",index_col=0)
test2 = pd.read_csv("./data/8-1 test2(std).csv",index_col=0)


In [6]:
x_train = train.drop("label",axis=1)
y_train = train["label"]
x_test =test1.drop("label",axis=1)
y_test = test1["label"]

In [7]:
x_train, y_train = SMOTE(random_state=4).fit_resample(x_train,y_train)
print(x_train.shape)
y_train.value_counts()

(4646, 10)


label
0.0    2323
1.0    2323
Name: count, dtype: int64

In [8]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

In [9]:
# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.2,0.3,0.4,0.5]

def get_eval_by_threshold(MDA_y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print("임곗값:", custom_threshold)
        get_clf_eval(MDA_y_test, custom_predict)
        print("---------------------------------------------------------")



In [10]:
# Roc Auc 그래프 값 확인 코드
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def roc_curve_plot(MDA_y_test, pred_proba_c1):
    fprs, tprs, thresholds = roc_curve(MDA_y_test, pred_proba_c1)
    
    plt.plot(fprs, tprs, label='ROC')
    plt.plot([0,1],[0,1], 'k--', label = 'Random')
    
    start, end= plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)'); plt.ylabel('TPR(Recall)')
    plt.legend()
    plt.show()

In [11]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()

lda_clf.fit(x_train, y_train)
lda_pred = lda_clf.predict(x_test)
lda_pred_proba = lda_clf.predict_proba(x_test)
print("---------------------------------------------------------") 
get_clf_eval(y_test, lda_pred)
get_eval_by_threshold(y_test , lda_pred_proba[:,1].reshape(-1,1), thresholds)
    

---------------------------------------------------------
오차행렬
[[2443  283]
 [  15   16]]
정확도: 0.8919, 정밀도 : 0.0535, 재현율:0.5161,F1 스코어:0.0970
ROC 스코어: 0.7062, PR score : 0.0331
임곗값: 0.1
오차행렬
[[1292 1434]
 [   6   25]]
정확도: 0.4777, 정밀도 : 0.0171, 재현율:0.8065,F1 스코어:0.0336
ROC 스코어: 0.6402, PR score : 0.0160
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1858  868]
 [   9   22]]
정확도: 0.6819, 정밀도 : 0.0247, 재현율:0.7097,F1 스코어:0.0478
ROC 스코어: 0.6956, PR score : 0.0208
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2123  603]
 [   9   22]]
정확도: 0.7780, 정밀도 : 0.0352, 재현율:0.7097,F1 스코어:0.0671
ROC 스코어: 0.7442, PR score : 0.0282
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2317  409]
 [  12   19]]
정확도: 0.8473, 정밀도 : 0.0444, 재현율:0.6129,F1 스코어:0.0828
ROC 스코어: 0.7314, PR score : 0.0316
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2443  283]
 [  15   16]]
정확도: 0.8919, 정밀도 : 0.0535, 재현율:0.5161,

In [12]:
# 1-2 디시전트리 
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)

dt_clf.fit(x_train, y_train)
dt_pred = dt_clf.predict(x_test)
dt_pred_proba = dt_clf.predict_proba(x_test)   

get_clf_eval(y_test, dt_pred)
get_eval_by_threshold(y_test , dt_pred_proba[:,1].reshape(-1,1), thresholds)
    


오차행렬
[[2595  131]
 [  25    6]]
정확도: 0.9434, 정밀도 : 0.0438, 재현율:0.1935,F1 스코어:0.0714
ROC 스코어: 0.5727, PR score : 0.0175
임곗값: 0.1
오차행렬
[[2595  131]
 [  25    6]]
정확도: 0.9434, 정밀도 : 0.0438, 재현율:0.1935,F1 스코어:0.0714
ROC 스코어: 0.5727, PR score : 0.0175
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2595  131]
 [  25    6]]
정확도: 0.9434, 정밀도 : 0.0438, 재현율:0.1935,F1 스코어:0.0714
ROC 스코어: 0.5727, PR score : 0.0175
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2595  131]
 [  25    6]]
정확도: 0.9434, 정밀도 : 0.0438, 재현율:0.1935,F1 스코어:0.0714
ROC 스코어: 0.5727, PR score : 0.0175
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2595  131]
 [  25    6]]
정확도: 0.9434, 정밀도 : 0.0438, 재현율:0.1935,F1 스코어:0.0714
ROC 스코어: 0.5727, PR score : 0.0175
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2595  131]
 [  25    6]]
정확도: 0.9434, 정밀도 : 0.0438, 재현율:0.1935,F1 스코어:0.0714
ROC 스코어: 0.5727, PR score : 0.0175
---------

In [13]:
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=52)

# key: 하이퍼파라미터 명; value: [후보값들] 을 리스트로 전달
param_grid = {
    'criterion': ['entropy', 'gini'],
            'max_depth': [2,4,6,8],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [4,6,8,10],
            'min_samples_split': [4,6,8,10]}
grid_search = GridSearchCV(tree
                        , param_grid = param_grid
                        , cv = 5 #교차검증시 fold 개수; n번의 교차검증; 데이터들이 돌아가면서 하나씩 검증(validation)에 사용되어 일반화하는 것; 데이터가 많지 않을 때 보통 쓰인다고 함
                        , n_jobs = -1)  #-1: 모든 코어 사용

grid_search.fit(x_train, y_train)

# 최선의 결과 출력
print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

dt_clf_best = DecisionTreeClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3], min_samples_split=values[4])
dt_clf_best.fit(x_train, y_train)
dt_best_pred = dt_clf_best.predict(x_test)
dt_best_pred_proba = dt_clf_best.predict_proba(x_test)   

print("디시전트리 파라미터적용") 
get_clf_eval(y_test, dt_best_pred)
get_eval_by_threshold(y_test ,dt_best_pred_proba[:,1].reshape(-1,1), thresholds)


최적 하이퍼 파라미터 :{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 4}, 최적 평균 정확도 :0.921
디시전트리 파라미터적용
오차행렬
[[2449  277]
 [  16   15]]
정확도: 0.8937, 정밀도 : 0.0514, 재현율:0.4839,F1 스코어:0.0929
ROC 스코어: 0.6911, PR score : 0.0307
임곗값: 0.1
오차행렬
[[2315  411]
 [  13   18]]
정확도: 0.8462, 정밀도 : 0.0420, 재현율:0.5806,F1 스코어:0.0783
ROC 스코어: 0.7149, PR score : 0.0291
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2402  324]
 [  14   17]]
정확도: 0.8774, 정밀도 : 0.0499, 재현율:0.5484,F1 스코어:0.0914
ROC 스코어: 0.7148, PR score : 0.0324
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2423  303]
 [  15   16]]
정확도: 0.8847, 정밀도 : 0.0502, 재현율:0.5161,F1 스코어:0.0914
ROC 스코어: 0.7025, PR score : 0.0313
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2436  290]
 [  16   15]]
정확도: 0.8890, 정밀도 : 0.0492, 재현율:0.4839,F1 스코어:0.0893
ROC 스코어: 0.6887, PR score : 0.0296
---------------------------------------

In [14]:
print("디시전트리") 
print(get_clf_eval(y_test, lda_pred))

디시전트리
오차행렬
[[2443  283]
 [  15   16]]
정확도: 0.8919, 정밀도 : 0.0535, 재현율:0.5161,F1 스코어:0.0970
ROC 스코어: 0.7062, PR score : 0.0331
None


In [15]:
# 1-3 랜덤포레스트 구현 
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)    

rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
rf_pred_proba = rf_clf.predict_proba(x_test)
print("랜덤포레스트") 
get_clf_eval(y_test, rf_pred)
get_eval_by_threshold(y_test , rf_pred_proba[:,1].reshape(-1,1), thresholds)



랜덤포레스트
오차행렬
[[2684   42]
 [  24    7]]
정확도: 0.9761, 정밀도 : 0.1429, 재현율:0.2258,F1 스코어:0.1750
ROC 스코어: 0.6052, PR score : 0.0410
임곗값: 0.1
오차행렬
[[2183  543]
 [  10   21]]
정확도: 0.7994, 정밀도 : 0.0372, 재현율:0.6774,F1 스코어:0.0706
ROC 스코어: 0.7391, PR score : 0.0289
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2463  263]
 [  13   18]]
정확도: 0.8999, 정밀도 : 0.0641, 재현율:0.5806,F1 스코어:0.1154
ROC 스코어: 0.7421, PR score : 0.0419
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2587  139]
 [  20   11]]
정확도: 0.9423, 정밀도 : 0.0733, 재현율:0.3548,F1 스코어:0.1215
ROC 스코어: 0.6519, PR score : 0.0333
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2655   71]
 [  24    7]]
정확도: 0.9655, 정밀도 : 0.0897, 재현율:0.2258,F1 스코어:0.1284
ROC 스코어: 0.5999, PR score : 0.0290
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2684   42]
 [  24    7]]
정확도: 0.9761, 정밀도 : 0.1429, 재현율:0.2258,F1 스코어:0.1750
ROC 스코어: 0.6052, PR score : 0.0410
--

In [16]:
# 하이퍼파라미터 Grid-Search
from sklearn.model_selection import GridSearchCV

    # 탐색 범위 정의

grid_search = {'criterion': ['entropy', 'gini'],
            'max_depth': [2,4,6,8],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [4,6,8,10],
            'min_samples_split': [4,6,8,10],
            'n_estimators': [20,40,60,80,100]}

model = RandomForestClassifier(random_state=52)
grid_search = GridSearchCV(estimator = model, param_grid = grid_search, 
                            cv = 5,verbose=0, n_jobs = -1)
grid_search.fit(x_train,y_train)

# 최선의 결과 출력
print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

rf_clf_best = RandomForestClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3],  min_samples_split=values[4],n_estimators=values[5])
rf_clf_best.fit(x_train, y_train)
rf_best_pred = rf_clf_best.predict(x_test)
rf_best_pred_proba = rf_clf_best.predict_proba(x_test)   

print("랜덤포레스트 파라미터적용") 
get_clf_eval(y_test, rf_best_pred)
get_eval_by_threshold(y_test ,rf_best_pred_proba[:,1].reshape(-1,1), thresholds)

최적 하이퍼 파라미터 :{'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}, 최적 평균 정확도 :0.965
랜덤포레스트 파라미터적용
오차행렬
[[2655   71]
 [  20   11]]
정확도: 0.9670, 정밀도 : 0.1341, 재현율:0.3548,F1 스코어:0.1947
ROC 스코어: 0.6644, PR score : 0.0549
임곗값: 0.1
오차행렬
[[1793  933]
 [   7   24]]
정확도: 0.6590, 정밀도 : 0.0251, 재현율:0.7742,F1 스코어:0.0486
ROC 스코어: 0.7160, PR score : 0.0220
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2235  491]
 [  11   20]]
정확도: 0.8179, 정밀도 : 0.0391, 재현율:0.6452,F1 스코어:0.0738
ROC 스코어: 0.7325, PR score : 0.0292
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2449  277]
 [  15   16]]
정확도: 0.8941, 정밀도 : 0.0546, 재현율:0.5161,F1 스코어:0.0988
ROC 스코어: 0.7073, PR score : 0.0336
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2578  148]
 [  19   12]]
정확도: 0.9394, 정밀도 : 0.0750, 재현율:0.3871,F1 스코어:0.1257
ROC 스코어: 0.6664, PR score : 0.0359
-------------------

In [17]:
# 1-4 로지스틱회귀
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()

lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)
lr_pred_proba = lr_clf.predict_proba(x_test)
print("로지스틱회귀") 
get_clf_eval(y_test, lr_pred)
get_eval_by_threshold(y_test , lr_pred_proba[:,1].reshape(-1,1), thresholds)
    

로지스틱회귀
오차행렬
[[2400  326]
 [  12   19]]
정확도: 0.8774, 정밀도 : 0.0551, 재현율:0.6129,F1 스코어:0.1011
ROC 스코어: 0.7467, PR score : 0.0381
임곗값: 0.1
오차행렬
[[1222 1504]
 [   5   26]]
정확도: 0.4527, 정밀도 : 0.0170, 재현율:0.8387,F1 스코어:0.0333
ROC 스코어: 0.6435, PR score : 0.0161
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1781  945]
 [   8   23]]
정확도: 0.6543, 정밀도 : 0.0238, 재현율:0.7419,F1 스코어:0.0460
ROC 스코어: 0.6976, PR score : 0.0205
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2075  651]
 [   9   22]]
정확도: 0.7606, 정밀도 : 0.0327, 재현율:0.7097,F1 스코어:0.0625
ROC 스코어: 0.7354, PR score : 0.0265
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2266  460]
 [  10   21]]
정확도: 0.8295, 정밀도 : 0.0437, 재현율:0.6774,F1 스코어:0.0820
ROC 스코어: 0.7543, PR score : 0.0332
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2400  326]
 [  12   19]]
정확도: 0.8774, 정밀도 : 0.0551, 재현율:0.6129,F1 스코어:0.1011
ROC 스코어: 0.7467, PR score : 0.0381
--

In [18]:
#로지스틱 회귀 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV


params = {'penalty': ['l2','l1','elasticnet','None'],
        'C':[0.01,0.1,1,3,5,10]}

grid_search = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
grid_search.fit(x_train,y_train)
print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))

a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

lr_clf_best = LogisticRegression(penalty= values[1], C=values[0])
lr_clf_best.fit(x_train, y_train)
lr_best_pred = lr_clf_best.predict(x_test)
lr_best_pred_proba = lr_clf_best.predict_proba(x_test)   

print("로지스틱회귀 파라미터적용") 
get_clf_eval(y_test, lr_best_pred)
get_eval_by_threshold(y_test ,lr_best_pred_proba[:,1].reshape(-1,1), thresholds)


최적 하이퍼 파라미어:{'C': 5, 'penalty': 'l2'}, 최적 평균 정확도:0.827
로지스틱회귀 파라미터적용
오차행렬
[[2400  326]
 [  12   19]]
정확도: 0.8774, 정밀도 : 0.0551, 재현율:0.6129,F1 스코어:0.1011
ROC 스코어: 0.7467, PR score : 0.0381
임곗값: 0.1
오차행렬
[[1225 1501]
 [   5   26]]
정확도: 0.4538, 정밀도 : 0.0170, 재현율:0.8387,F1 스코어:0.0334
ROC 스코어: 0.6440, PR score : 0.0161
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1782  944]
 [   8   23]]
정확도: 0.6547, 정밀도 : 0.0238, 재현율:0.7419,F1 스코어:0.0461
ROC 스코어: 0.6978, PR score : 0.0205
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2076  650]
 [   9   22]]
정확도: 0.7610, 정밀도 : 0.0327, 재현율:0.7097,F1 스코어:0.0626
ROC 스코어: 0.7356, PR score : 0.0265
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2266  460]
 [  10   21]]
정확도: 0.8295, 정밀도 : 0.0437, 재현율:0.6774,F1 스코어:0.0820
ROC 스코어: 0.7543, PR score : 0.0332
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2400  326]
 [  12   19]]
정확도: 0.8774, 정밀도 : 0.0551, 